In [11]:
# Импорт модулей
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import time

In [12]:
# Создание Спарк сессии
spark = SparkSession.builder.appName('odd_sum').getOrCreate()

In [13]:
# Чтение данных из источника rate
streaming_df = spark.readStream.format('rate').option("rowsPerSecond", 1).load()

In [14]:
# Фильтрация нечётных значений
odd_df = streaming_df.filter(col('value') % 2 == 1)

In [15]:
# Вычисление суммы нечётных значений
sum_df = odd_df.groupBy().sum('value').withColumnRenamed('sum(value)', 'sum_odds')

In [16]:
# Определение функции для обработки каждой партии информации
def process_batch(df, epoch_id):
    print(f"Batch {epoch_id}:")
    df.show()

In [20]:
# Запись результатов
query = sum_df.writeStream.outputMode("complete").foreachBatch(process_batch).start()

In [21]:
# Ожидание завершения потока в течение 10 секунд
time.sleep(10)

Batch 2:
+--------+
|sum_odds|
+--------+
|       1|
+--------+

Batch 3:
+--------+
|sum_odds|
+--------+
|       1|
+--------+

Batch 4:
+--------+
|sum_odds|
+--------+
|       4|
+--------+

Batch 5:
+--------+
|sum_odds|
+--------+
|       4|
+--------+

Batch 6:
+--------+
|sum_odds|
+--------+
|       9|
+--------+

Batch 7:
+--------+
|sum_odds|
+--------+
|       9|
+--------+

Batch 8:
+--------+
|sum_odds|
+--------+
|      16|
+--------+

Batch 9:
+--------+
|sum_odds|
+--------+
|      16|
+--------+

Batch 10:
+--------+
|sum_odds|
+--------+
|      25|
+--------+

Batch 11:
+--------+
|sum_odds|
+--------+
|      25|
+--------+



In [19]:
# Остановка запроса
query.stop()